In [6]:
import pandas as pd
import numpy as np
from transformers import AutoModel, AutoTokenizer
import faiss
import torch
from tqdm.auto import trange
from string import punctuation
from spacy.lang.en import stop_words
import spacy
import pickle
import re
import torch.functional as F

def prepare_text(s):
    return re.sub(" +", " ",s.lower().replace('_',' ').strip())

def parse_request_csqa(x):
    return f"{x['question_state']} {x['question']}"

def parse_request_dream(x):
    return f"{x['dialog']} {x['question']} {x['choice_0']} {x['choice_1']}  {x['choice_2']}" # 

def match_pairs(c1,c2):
    state = list(c1)+list(c2)
    idx = np.unique(state, return_index=True)[1]
    return [state[i] for i in sorted(idx)][::-1]

def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

    
def cls_pooling(model_output):
    return model_output[0][:, 0]

@torch.no_grad()
def create_embeds(texts,model,tokenizer,batch_size=64,max_length=1024):
    features = []
    texts = [prepare_text(x) for x in texts]
    for i in trange(len(texts) // batch_size + 1):
        encs = tokenizer(
            texts[i*batch_size:(i+1)*batch_size],
            truncation=True,
            padding='longest',
            max_length=max_length,
            return_tensors='pt'
        )
        encs['input_ids'] = encs['input_ids'].cuda()
        encs['attention_mask'] = encs['attention_mask'].cuda()
        if 'token_type_ids' in encs.keys():
            encs['token_type_ids'] = encs['token_type_ids'].cuda()
        model_output = model(**encs)
        embeds = cls_pooling(model_output)#mean_pooling(model_output, encs['attention_mask'])
        embeds = torch.nn.functional.normalize(embeds, p=2, dim=1).detach().cpu().numpy()
        features.append(embeds)
    return np.concatenate(features,dtype=np.float32)

class TextProcessor():
    def __init__(self, kn_base='en_core_web_sm'):
        self.nlp = spacy.load(kn_base)
        self.stop_wrods = stop_words.STOP_WORDS | set(punctuation) | set('123456789')
        
    def prepare_text(self, s):
        s = s.lower().replace('_',' ').strip()
        for p in punctuation:
            s = s.replace(p,' ')
        return re.sub(" +", " ",s)
    
    def precess_text(self,s):
        s = self.prepare_text(s)
        doc = self.nlp(s)
        lemms = [token.lemma_ for token in doc]
        lemms = [token for token in lemms if token not in self.stop_wrods]
        return lemms
    
    def process_batch(self,texts,verbose=True):
        iterator = tqdm(texts) if verbose else texts
        pool = mp.Pool(mp.cpu_count())
        prep_texts = pool.map(self.precess_text,iterator)
        return prep_texts

def rank_by_idf(texts,idf_matrix,topk=5):
    idx = np.unique(texts, return_index=True)[1]
    texts = [texts[i] for i in sorted(idx)]
    idfs = [(idf_matrix[x] if x in idf_matrix else 0) for x in texts]
    return [texts[i] for i in np.argsort(idfs)[:topk]]

def setup_index(path):   
    features = pickle.load(open(path,'rb'))
    index = faiss.IndexFlatIP(features.shape[1]) #use faiss.IndexFlatL2 for l2 distance
    faiss.normalize_L2(features)
    index.add(features)
    res = faiss.StandardGpuResources()
    index = faiss.index_cpu_to_gpu(res, 0, index)
    return index

def get_retrivers():
    encoder = AutoModel.from_pretrained('BAAI/bge-small-en-v1.5').cuda()
    tokenizer = AutoTokenizer.from_pretrained('BAAI/bge-small-en-v1.5')
    return encoder,tokenizer

class EntityLinker():
    def __init__(self, kg, text_proc, prepared2concept, prepared_maper, clear_nodes):
        self.kg = kg
        self.text_proc = text_proc
        self.prepared2concept = prepared2concept
        self.prepared_concepts = prepared_maper
        self.cleared_nodes = clear_nodes
        
    def link_sentence(self,s):
        candidates = []
        s = self.text_proc.precess_text(s)
        s_st = set(s)
        for i,k in enumerate(self.kg):
            if len(k) == 0:
                continue
            elif len(k) == 1:
                if k[0] in s_st:
                    candidates.append(i)
            else:
                if all(x in s_st for x in k):
                    idxes = [s.index(x) for x in k]
                    if all(idxes[j+1] - idxes[j] == 1 for j in range(len(k)-1)):
                           candidates.append(i)
        return candidates
    
    def batch_linking(self,texts,verbose=True):
        iterator = tqdm(texts) if verbose else texts
        pool = mp.Pool(mp.cpu_count())
        prep_texts = pool.map(self.link_sentence,iterator)
        return prep_texts
    
    def get_concepts(self,ids):
        ids = [self.kg[i] for i in ids] # from index to lemms
        ids = [' '.join(x) for x in ids]
        return [self.prepared2concept[i] for i in ids] # from lemms to concepts
    
    
class SampleNodesMatcher():
    def __init__(self, encoder, tokenizer, index, idf_matrix, maper, linker):
        self.encoder = encoder
        self.tokenizer = tokenizer
        self.index = index
        self.idf_matrix = idf_matrix
        self.linker = linker
        self.maper = maper
    
    def __call__(self, sample,type='csqa',use_pretty=False):
        if type == 'csqa':
            req = parse_request_csqa(sample)
        else:
            req = parse_request_dream(sample)
        if use_pretty:
            maper = self.linker.cleared_nodes
        else:
            maper = self.maper
        embeds = create_embeds([req], self.encoder, self.tokenizer,batch_size=2,max_length=512)
        faiss.normalize_L2(embeds)
        D, I = self.index.search(embeds,k=7)
        I = [maper[i] for i in I[0]]
        
        linker_cand = self.linker.link_sentence(req)
        linker_cand = self.linker.get_concepts(linker_cand)
        linker_cand = rank_by_idf(linker_cand,self.idf_matrix[type])
        if type == 'csqa':
            return match_pairs(linker_cand,I)
        else:
            choice_embeds = create_embeds([sample['choice_0'], sample['choice_1'], sample['choice_2']], self.encoder, self.tokenizer,batch_size=16,max_length=512)
            faiss.normalize_L2(choice_embeds)
            D, I_q = self.index.search(choice_embeds,k=1)
            choice_encs = [self.linker.cleared_nodes[i] for i in I_q[:,0]]
            return match_pairs(linker_cand,I), choice_encs

def setup_processor(linker_path="entity_linker.pkl", embeds_path='faiss_embeds.pkl', idf_path='idf_matrix.pkl'):
    linker = pickle.load(open(linker_path,'rb'))
    index = setup_index(embeds_path)
    idf_matrix = pickle.load(open(idf_path,'rb'))
    encoder,tokenizer = get_retrivers()
    return SampleNodesMatcher(
    encoder, tokenizer, index, idf_matrix, linker.maper_inv, linker
    )

In [7]:
import os
import gc
import pandas as pd
import numpy as np
import pickle
import pytorch_lightning as pl
from src import CustomGraphMultipleChoiceEncoder, GraphProcessorV2
from sklearn.model_selection import train_test_split, StratifiedKFold, GroupKFold
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch_geometric.data import Batch, Data
from scipy.stats import rankdata
from tqdm.auto import tqdm, trange
from sklearn.metrics import accuracy_score
from transformers import get_cosine_schedule_with_warmup,get_linear_schedule_with_warmup, AutoModel, AutoTokenizer, AutoModelForMultipleChoice, AutoConfig
import wandb
import random
pl.seed_everything(56)
os.environ["TOKENIZERS_PARALLELISM"] = "false"

class CFG:
    class data:
        train_path = 'csqa_data/train.statement.jsonl'
        dev_path = 'csqa_data/dev.statement.jsonl'
        test_path = 'csqa_data/test.statement.jsonl'
        dream_train_path = 'data/train.statement.jsonl'
        dream_dev_path = 'data/dev.statement.jsonl'
        dream_test_path = 'data/test.statement.jsonl'
        dream_train_concept_path = 'prep_data/train_choices.csv'
        dream_dev_concept_path = 'prep_data/vall_choices.csv'
        dream_test_concept_path = 'prep_data/test_choices.csv'
        tokenizer = "microsoft/deberta-v3-large"
        graph_path = 'graph.pkl'
        embeds_path = 'deberta_large_embedsV2.npy'
        maper_path = 'csqa_data/maper_inv.pkl'
        concepts_lst = 'maper_v2.pkl'
        #concepts_lst = 'csqa_data/maper.pkl'
        dop_concepts_csqa_path = 'csqa_data/all_candsV3.pkl'
        dop_concepts_dream_path = 'dream_all_candidates.pkl'
        num_workers = 0
        pad_to_nodes = 64
        nfolds = 5
        batch_size = 4
        use_prefix = True
        max_length = 384 
        seed = 56
    class model:
        model = "microsoft/deberta-v3-large"
        optim = torch.optim.AdamW
        use_only_encoder = False
        use_tkds = True
        grad_acum_steps = 1
        not_padding_token = False
        torch_dtype = None
        scheduler = 'cosine'
        warmup_steps = 0.0 #0.25
        num_labels = 1
        label_smoothing = 0.0
        lr = 1e-5
        lr_gnn = 1e-5
        lr_fn = 1e-4
        cls_drop_type = None
        cls_drop = 0.0
        pool = 'attention'
        max_epoches = 5
        no_decay = ["bias", "LayerNorm.bias", "LayerNorm.weight"]
        turn_off_drop = True
        num_cycles = 0.5
        eps = 1e-7
        weight_decay = 0.0
        weight_decay_fn = 0.0
        betas = (0.9, 0.999)
        use_lora = False
        class gnn:
            pool_type = 'no'
            num_nodes = 64
            max_nodes = 64
            edge_dim = 17
            blocks = [
                {'block_type':'gatv2','act':nn.GELU(),'block_norm':None,'drop_p':0.0,'input_dim':1024,'out_dim':1024,'heads':2},
                {'block_type':'gatv2','act':nn.GELU(),'block_norm':None,'drop_p':0.0,'input_dim':2048,'out_dim':1024,'heads':1},
                {'block_type':'mlp','act':nn.Identity(),'block_norm':None,'drop_p':0.0,'input_dim':1024,'out_dim':1024},
            ]

    seed = 56
    fold_number = 0

def set_wandb_cfg():
    config = {}
    for k,v in CFG.model.__dict__.items():
        if '__' not in k:
            config[k] = v
    for k,v in CFG.data.__dict__.items():
        if '__' not in k:
            config[k] = v
    config['fold_number'] = CFG.fold_number
    return config

class CsqaDataset(Dataset):
    def __init__(self, tokenizer, graph_miner, concepts):
        super().__init__()
        self.cfg = CFG.data
        self.tokenizer = tokenizer
        self.graph_miner = graph_miner
        self.concepts = concepts
        
    def __len__(self):
        return len(self.data)

    def prepare_prompt(self, question, concepts, answer):
        s = f"Question: {question} "
        return s, f"Answer: {answer}"
    
    def parse_answer(self, answer, padd_node=160892):
        answer = answer.replace(' ','_')[:]
        if answer in self.concepts:
            res = self.concepts[answer]
            if res == -100:
                return padd_node
            return res
        else:
            return padd_node
    
    def get_row_encodes(self, row):
        if self.cfg.use_prefix:
            prompts = [
                self.prepare_prompt(row['question'], row['concepts'], i)
                for i in [row['choice_A'],row['choice_B'],row['choice_C'],row['choice_D'],row['choice_E']]
            ]
        else:
            prompts = [
                [row['question'],i] # old ver
                for i in [row['choice_A'],row['choice_B'],row['choice_C'],row['choice_D'],row['choice_E']]
            ]
        
        encodes = [
            self.tokenizer.encode_plus(
                question,answer,
                max_length=self.cfg.max_length,
                truncation='only_first',
                padding='max_length',
                return_tensors='pt'
            )
            for question,answer in prompts
        ]
        answers = [x for x in [row['choice_A'],row['choice_B'],row['choice_C'],row['choice_D'],row['choice_E']]]
        graph_encs = [
            Data(**self.graph_miner.process([self.parse_answer(x) for x in ([a]+row['concepts'])]))
            for a in answers
        ]
        return {
            'input_ids':torch.stack([x.input_ids.squeeze(0) for x in encodes]).unsqueeze(0),
            'attention_mask':torch.stack([x.attention_mask.squeeze(0) for x in encodes]).unsqueeze(0),
            'token_type_ids':torch.stack([x.token_type_ids.squeeze(0) for x in encodes]).unsqueeze(0),
            'graph_encodes': Batch.from_data_list(graph_encs),
        }

class DreamDataset(Dataset):
    def __init__(self, tokenizer, graph_miner, concepts):
        super().__init__()
        self.cfg = CFG.data
        self.tokenizer = tokenizer
        self.graph_miner = graph_miner
        self.concepts = concepts
        
    def __len__(self):
        return len(self.data)

    def prepare_prompt(self, dialog, question, concepts, answer):
        s = f"Dialog: {dialog}[SEP]Question: {question} "
        return s, f"Answer: {answer}"
    
    def parse_answer(self, answer, padd_node=160892):
        answer = answer.replace(' ','_')[:]
        if answer in self.concepts:
            res = int(self.concepts[answer])
            if res == -100:
                return padd_node
            return res
        else:
            return padd_node

    def parse_dialog(self, dialog):
        return '[SEP]'.join(dialog)
    
    def get_row_encodes(self, row):
        dialog = self.parse_dialog(row['dialog'])
        if self.cfg.use_prefix:
            prompts = [
                self.prepare_prompt(dialog, row['question'], row['concepts'], i)
                for i in [row['choice_0'],row['choice_1'],row['choice_2']]
            ]
        else:
            prompts = [
                [row['question'],i] # old ver
                for i in [row['choice_0'],row['choice_1'],row['choice_2']]
            ]
        
        encodes = [
            self.tokenizer.encode_plus(
                question,answer,
                max_length=self.cfg.max_length,
                truncation='only_first',
                padding='max_length',
                return_tensors='pt'
            )
            for question,answer in prompts
        ]
        answers = [x for x in [row['choice_0'],row['choice_1'],row['choice_2']]]
        graph_encs = [
            Data(**self.graph_miner.process([self.parse_answer(x) for x in ([a]+row['concepts'])]))
            for a in answers
        ]
        return {
            'input_ids':torch.stack([x.input_ids.squeeze(0) for x in encodes]).unsqueeze(0),
            'attention_mask':torch.stack([x.attention_mask.squeeze(0) for x in encodes]).unsqueeze(0),
            'token_type_ids':torch.stack([x.token_type_ids.squeeze(0) for x in encodes]).unsqueeze(0),
            'graph_encodes': Batch.from_data_list(graph_encs),
        }


def build_datasets(graph_miner_path, concepts_path,task='csqa'):
    graph_miner = pickle.load(open(graph_miner_path,'rb'))
    concepts_path = pickle.load(open(concepts_path,'rb'))
    tokenizer = AutoTokenizer.from_pretrained(CFG.model.model)
    if task == 'csqa':
        return CsqaDataset(tokenizer, graph_miner, concepts_path), tokenizer
    else:
        return DreamDataset(tokenizer, graph_miner, concepts_path), tokenizer

@torch.no_grad()
def infer_sample(sample,task='csqa',chekpoint=None, device='cuda', return_cache=False):
    model = CustomGraphMultipleChoiceEncoder(CFG.model).to(device).eval()
    if chekpoint:
        model.load_state_dict(torch.load(chekpoint))
    processor = setup_processor()
    dataset, tokenizer = build_datasets(graph_miner_path="graph_miner.pkl", concepts_path = "maper_v2.pkl", task=task)

    outs = processor(sample, type=task)
    if task == 'csqa':
        sample['concepts'] = outs
    else:
        sample['concepts'] = outs[0]
        sample['choice_0'] = outs[1][0]
        sample['choice_1'] = outs[1][1]
        sample['choice_2'] = outs[1][2]

    encodes = dataset.get_row_encodes(sample)
    for k in encodes:
        encodes[k] = encodes[k].to(device)
    logits = model(**encodes).logits.detach().cpu()

    if not return_cache:
        return logits.tolist()
    else:
        chache = {
            'model': model,
            'tokeinzer': tokenizer,
            'dataset': dataset,
            'processor': processor,
        }
        return logits.tolist(), chache

Seed set to 56


In [8]:
def make_df(path,is_test=False):
    data = pd.read_json(path,lines=True)
    df = pd.DataFrame()
    
    df['question_state'] = data['question'].apply(lambda x: x['question_concept'])
    df['id'] = data['id']
    df['question'] = data['question'].apply(lambda x: x['stem'])
    df['choice_A'] = data['question'].apply(lambda x: x['choices'][0]['text'])
    df['choice_B'] = data['question'].apply(lambda x: x['choices'][1]['text'])
    df['choice_C'] = data['question'].apply(lambda x: x['choices'][2]['text'])
    df['choice_D'] = data['question'].apply(lambda x: x['choices'][3]['text'])
    df['choice_E'] = data['question'].apply(lambda x: x['choices'][4]['text'])
    
    if not is_test:
        df['label'] = data['answerKey'].map({'A':0,'B':1,'C':2,'D':3,'E':4})
    else:
        df['label'] = -100
    return df
df = make_df('csqa_data/train.statement.jsonl')

In [9]:
def prepare_text(s):
    return re.sub(" +", " ",s.lower().replace('_',' ').strip())

def parse_dialog(sentences):
    return '; '.join(sentences)

def parse_request(x):
    return f"{x['question_state']} {x['question']}"

def make_dream_df(path, is_test=False):
    data = pd.read_json(path,lines=True)
    df = []
    
    for i in range(len(data)):
        row = data.iloc[i]
        dialog = parse_dialog(row[0])
        for ques in row[1]:
            df.append({
                'dialog_id': i,
                'dialog': dialog,
                'question': ques['question'],
                'choice_0': ques['choice'][0],
                'choice_1': ques['choice'][1],
                'choice_2': ques['choice'][2],
                'labels': -100 if is_test else ques['choice'].index(ques['answer'])
            })
    return pd.DataFrame(df)

dream_df = make_dream_df('data/train.statement.jsonl')

In [10]:
row = dream_df.iloc[0]

In [11]:
infer_sample(row,task='dream')

config.json:   0%|          | 0.00/743 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/133M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

...

/usr/local/lib/python3.11/dist-packages/transformers/convert_slow_tokenizer.py:558: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

/tmp/ipykernel_386/3550083374.py:245: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample['concepts'] = outs[0]
/tmp/ipykernel_386/3550083374.py:245: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample['concepts'] = outs[0]
/tmp/ipykernel_386/3550083374.py:246: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample['choice_0'] = outs[1][0]
/tmp/ipykernel_386/3550083374.py:247: SettingWithCopyWarning: 
A value is trying to be set on a c

[[0.5139333605766296, 0.6147091388702393, 0.4759402573108673]]

In [18]:
sample=df.iloc[0]; task='csqa'; chekpoint=None; device='cuda'; return_cache=False

model = CustomGraphMultipleChoiceEncoder(CFG.model).to(device).eval()
if chekpoint:
    model.load_state_dict(torch.load(chekpoint))
processor = setup_processor()
dataset, tokenizer = build_datasets(graph_miner_path="graph_miner.pkl", concepts_path = "maper_v2.pkl")
    
outs = processor(sample, type=task)
if task == 'csqa':
    sample['concepts'] = outs
else:
    sample['concepts'] = outs[0]
    sample['choice_0'] = outs[1][0]
    sample['choice_1'] = outs[1][1]
    sample['choice_2'] = outs[1][2]
    
encodes = dataset.get_row_encodes(outs)
for k in batch:
    batch[k] = batch[k].to(device)
logits = model(**batch).logits

  0%|          | 0/1 [00:00<?, ?it/s]

TypeError: list indices must be integers or slices, not str

In [12]:
row = df.iloc[0]

In [14]:
row['question_state'] = ''
row['question'] = 'Where on a river can you hold a cup upright to catch water on a sunny day?'

/tmp/ipykernel_1132/2296104930.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  row['question_state'] = ''
/tmp/ipykernel_1132/2296104930.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  row['question'] = 'Where on a river can you hold a cup upright to catch water on a sunny day?'


In [11]:
processor = setup_processor()

In [19]:
processor(row,use_pretty=True)

  0%|          | 0/1 [00:00<?, ?it/s]

['have cup in hand',
 'holds rainwater',
 'cup water',
 'holds drink',
 'holding water',
 'holds water',
 'hold cup',
 'catch',
 'hold',
 'day',
 'upright',
 'water']

In [20]:
1

1

In [36]:
dataset, tokenizer = build_datasets(graph_miner_path="graph_miner.pkl", concepts_path = "maper_v2.pkl")

/usr/local/lib/python3.11/dist-packages/transformers/convert_slow_tokenizer.py:558: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [38]:
outs = processor(sample, type=task)
if task == 'csqa':
    sample['concepts'] = outs
else:
    sample['concepts'] = outs[0]
    sample['choice_0'] = outs[1][0]
    sample['choice_1'] = outs[1][1]
    sample['choice_2'] = outs[1][2]
    
encodes = dataset.get_row_encodes(sample)
for k in encodes:
    encodes[k] = encodes[k].to(device)
    
logits = model(**encodes).logits

  0%|          | 0/1 [00:00<?, ?it/s]

/tmp/ipykernel_196/2743646351.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample['concepts'] = outs


In [39]:
logits

tensor([[0.7954, 0.9044, 0.9010, 0.8943, 0.9010]], device='cuda:0',
       grad_fn=<ViewBackward0>)